In [ ]:
# Import Libraries
import time

from ImportData2D import get_label, getFilename, get_file_name
from stlSlicer import getSlicer, slicecoor, rotatestl
from imgSave import saveplot
import numpy as np

np.random.seed(125)


In [ ]:
label = getLabel("Occ_Sum", "median", True, False,False)
print(type(label[0]))
label = getLabel("Occ_Sum", "median", True, True,False)
print(type(label[0][0]))

In [ ]:
import os
test_name = "prep_test.tfrecords"
test_name = os.path.join("data", test_name)
print(test_name)

In [ ]:
import os
A = './data/preparation_181_data_test_eval_address.txt'
print(os.path.abspath(A))
print(os.path.isfile(A))

In [ ]:
example_grouped_address = ['/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_0_84140','/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_1_95138_0','/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_1_304138_45']
grouped_address = [1,2,3]
# define an empty list
train_address = []

# open file and read the content in a list
with open(A, 'r') as filehandle:  
    for line in filehandle:
        # remove linebreak which is the last character of the string
        current_name = line[:-1]
        # print(current_name)
        for i, name in enumerate(example_grouped_address):
            
            if current_name in name:
                train_address.append(grouped_address[i])
                grouped_address.remove(grouped_address[i])
                example_grouped_address.remove(example_grouped_address[i])
                

In [ ]:
print(train_address)

In [ ]:
import ast
B = ast.literal_eval(error_file_names[0])
print(B[0])

In [ ]:
train_address = ['abc_0_0','abc_0_90','abc_0_45','abc_0_135','abc_1_0','abc_1_45','abc_1_90','abc_1_70']

In [ ]:
train_address.sort()
print([s for s in train_address if 'abc_0' in s])

In [ ]:
import tensorflow as tf
import csv
A = {'learning_rate':0.00009123,'keep_prob':0.25, 'activation':tf.nn.relu, 'channels':[16,16,32]}
# A = {'a':'b','c':'d'}

with open("output"+".csv", "w") as csvfile:
    writer = csv.writer(csvfile)
    for key, val in A.items():
        writer.writerow([key, val])


In [1]:
from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
# You can add the parameter data_home to wherever to where you want to download your data
import tensorflow as tf
import numpy as np
((train_data, train_labels),(eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()
train_data = train_data / np.float32(255)
train_data = train_data.reshape(train_data.shape[0], -1)
print(train_data.shape)
train_labels = train_labels.astype(np.int32)  # not required
print(train_labels.shape)
# test_size: what proportion of original data is used for test set
train_img, test_img, train_lbl, test_lbl = train_test_split(
    train_data, train_labels, test_size=1/7.0, random_state=0)
print(train_img.shape)

(60000, 784)
(60000,)
(51428, 784)


In [3]:
# Standardizing the Data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(train_img)

# Apply transform to both the training set and the test set.
train_img = scaler.transform(train_img)
test_img = scaler.transform(test_img)

In [4]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
pca = PCA(.95)
pca.fit(train_img)
pca.n_components_
train_img = pca.transform(train_img)
test_img = pca.transform(test_img)

# all parameters not specified are set to their defaults
# default solver is incredibly slow thats why we change it
# solver = 'lbfgs'
logisticRegr = LogisticRegression(solver = 'lbfgs')

logisticRegr.fit(train_img, train_lbl)

# Returns a NumPy Array
# Predict for One Observation (image)
logisticRegr.predict(test_img[0].reshape(1,-1))

# Predict for Multiple Observations (images) at Once
logisticRegr.predict(test_img[0:10])

/home/pasin/anaconda3/envs/ps36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/pasin/anaconda3/envs/ps36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/pasin/anaconda3/envs/ps36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/pasin/anaconda3/envs/ps36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/pasin/anaconda3/envs/ps36/lib/python3.6/site-packages/sklearn/linear_model/logi

array([3, 6, 6, 6, 0, 3, 6, 2, 5, 6], dtype=int32)

In [5]:
score = logisticRegr.score(test_img, test_lbl)
print(score)

0.922071861875875


In [1]:
import tensorflow as tf
# tf.enable_eager_execution()
import numpy as np

# Fixed parameter
numdegree = 4  # Number of rotations
image_height = 240  # Used for cropping
image_width = 360  # Used for cropping


# Import tfrecord to dataset
def deserialize(example):
    feature = {'label': tf.FixedLenFeature([], tf.int64)}
    for i in range(numdegree):
        feature['img' + str(i)] = tf.FixedLenFeature([], tf.string)
    return tf.parse_single_example(example, feature)


def decode(data_dict):
    if numdegree != 4:
        raise Exception('Number of degree specified is not compatible, edit code')
    # Create initial image, then stacking it
    image_decoded = []

    # Stacking the rest
    for i in range(0, numdegree):
        img = data_dict['img' + str(i)]
        file_decoded = tf.image.decode_png(img, channels=1)
        file_cropped = tf.squeeze(tf.image.resize_image_with_crop_or_pad(file_decoded, image_height, image_width))
        image_decoded.append(file_cropped)

    image_stacked = tf.stack([image_decoded[0], image_decoded[1], image_decoded[2], image_decoded[3]], axis=2)
    image_stacked = tf.cast(image_stacked, tf.float32)
    label = data_dict['label']
    # output = (image_stacked, label)
    # return {'images': image_stacked, 'label': label}  # Output is [Channel, Height, Width]
    return image_stacked, label


def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def get_data_from_path(data_path):
    dataset = tf.data.TFRecordDataset(data_path)
    #print(dataset)
    dataset = dataset.map(deserialize)
    dataset = dataset.map(decode)
    
    max_elems = np.iinfo(np.int64).max
    dataset = dataset.batch(1000, drop_remainder=False)
    whole_dataset_tensors = tf.data.experimental.get_single_element(dataset)
    with tf.Session() as sess:
        whole_dataset_arrays = sess.run(whole_dataset_tensors)
    return whole_dataset_arrays
    
    '''
    iterator = dataset.make_one_shot_iterator()
    
    next_example, next_label = iterator.get_next()
    with tf.Session() as sess:
        example = sess.run(next_example)
        label = sess.run(next_label)
    return example, label
    '''

In [3]:
data_path = './data/preparation_181_data_Taper_Sum_median_eval.tfrecords'
arrays = get_data_from_path(data_path)
# example, label = get_data_from_path(data_path)
# print(example[0])
print((arrays[1]))

[ 8  8  8  8  8  8 10  6 10  8  8  8  6 10  6 10  6  8  8 10  8  6  8  8
  8  6  8  6  6 10 10  8  6  8  8  8  8  8  8  8  6  8  8  4  6  6  8  8
  8  6  6  6  8 10  8  6  6  8  8  6  6  6  4  8  8  8  8  8  8  8 10  8
 10  7  6 10  8  8  8  8  6  8  8  8  8 10  8 10  6 10  6  8  8  8 10  4
  8  8  8  8 10  6  8  8  8  8  8  6 10  8  8 10 10  9  6  8  6  9  8 10
  6  8  6  6  8 10  8  8  8  8  8  6  6 10  6  6  6  8  8  6  6  8 10 10
  8]


In [ ]:
ex, label = get_data_from_path(data_path)
print(label)

In [ ]:
ex, label = get_data_from_path(data_path)
print(label)